### Business Problem
On-time deliveries play an important role in ecommerce success. One of the biggest reason for Amazon's massive growth and success is their quickest and on-time delivery promise. The supply chain optimization plays a crucial role in making sure that the orders are delivered quickers and on promised time. <br/>
<b>I will develop a classification model to predict if a delivery promise can be met.</b> <br/>
This model will help business/merchants to optimize their supply chain and provide a better estimate for deliveries to customer. 


### Dataset
This dataset contains public ecommerce dataset of orders and deliveries made at store Olist store. <br/>Olist is the largest department store in Brazil. Olist connects small businesses
from all over Brazil. Merchants can sell their products through Olist store and ship them directly to customers using Olist logistics partners.

The dataset also contains the satisfaction survey customers filled either post receiving their delivery or their delivery was delayed.</br>

<b>The architecture of the dataset is shown below</b>
<img width="500" height="500" src="HRhd2Y0.png"/>

ref - https://www.kaggle.com/olistbr/brazilian-ecommerce

In [55]:
#imports
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

In [44]:

def print_float(val):
    print("{:.2f}".format(val))


In [11]:
#path
_path = "../ecommerce/"

In [81]:
_file_path_order_dataset = 'olist_orders_dataset.csv'
_file_path_cust = 'olist_customers_dataset.csv'
_file_path_geolocation = 'olist_geolocation_dataset.csv'
_file_path_items = 'olist_order_items_dataset.csv'
_file_path_seller = 'olist_sellers_dataset.csv'

In [82]:

df_orders = pd.read_csv(_path + _file_path_order_dataset)
df_cust = pd.read_csv(_path + _file_path_cust)
df_geo = pd.read_csv(_path + _file_path_geolocation)
df_item = pd.read_csv(_path + _file_path_items)
df_seller = pd.read_csv(_path + _file_path_seller)

seller_id  seller_zip_code_prefix  \
0  3442f8959a84dea7ee197c632cb2df15                   13023   
1  d1b65fc7debc3361ea86b5f14c68d2e2                   13844   
2  ce3ad9de960102d0677a81f5d0bb7b2d                   20031   
3  c0f3eea2e14555b6faeea3dd58c1b1c3                    4195   
4  51a04a8a6bdcb23deccc82b0b80742cf                   12914   

         seller_city seller_state  
0           campinas           SP  
1         mogi guacu           SP  
2     rio de janeiro           RJ  
3          sao paulo           SP  
4  braganca paulista           SP

#### Null Checks

In [83]:
print(df_cust.isnull().sum())
print(df_orders.isnull().sum())
print(df_geo.isnull().sum())
print(df_item.isnull().sum())
print(df_seller.isnull().sum())


customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64


In [84]:
df_orders.shape

(99441, 8)

In [91]:
df_orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [90]:
df_item.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [102]:
more_than_one_order = set(df_item.groupby('order_id').filter(lambda x:len(x)>1)['order_id'].tolist())

In [103]:
len(more_than_one_order)
df_item.drop

9803

In [96]:
df_orders.merge(df_item, how='left', on='order_id', suffixes=('_o','_r'))

order_id                       customer_id  \
0       e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1       53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2       47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3       949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4       ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
...                                  ...                               ...   
113420  63943bddc261676b46f01ca7ac2f7bd8  1fca14ff2861355f6e5f14306ff977a7   
113421  83c1379a015df1e13d02aae0204711ab  1aa71eb042121263aafbe80c1b562c9c   
113422  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
113423  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
113424  66dea50a8b16d9b4dee7af250b4be1a5  edb027a75a1449115f6b43211ae02a24   

       order_status order_purchase_timestamp    order_approved_at  \
0         delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1         delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2         delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3         delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4         delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   
...             ...                      ...                  ...   
113420    delivered      2018-02-06 12:58:58  2018-02-06 13:10:37   
113421    delivered      2017-08-27 14:46:43  2017-08-27 15:04:16   
113422    delivered      2018-01-08 21:28:27  2018-01-08 21:36:21   
113423    delivered      2018-01-08 21:28:27  2018-01-08 21:36:21   
113424    delivered      2018-03-08 20:57:30  2018-03-09 11:20:28   

       order_delivered_carrier_date order_delivered_customer_date  \
0               2017-10-04 19:55:00           2017-10-10 21:25:13   
1               2018-07-26 14:31:00           2018-08-07 15:27:45   
2               2018-08-08 13:50:00           2018-08-17 18:06:29   
3               2017-11-22 13:39:59           2017-12-02 00:28:42   
4               2018-02-14 19:46:34           2018-02-16 18:17:02   
...                             ...                           ...   
113420          2018-02-07 23:22:42           2018-02-28 17:37:56   
113421          2017-08-28 20:52:26           2017-09-21 11:24:17   
113422          2018-01-12 15:35:03           2018-01-25 23:32:54   
113423          2018-01-12 15:35:03           2018-01-25 23:32:54   
113424          2018-03-09 22:11:59           2018-03-16 13:08:30   

       order_estimated_delivery_date  order_item_id  \
0                2017-10-18 00:00:00            1.0   
1                2018-08-13 00:00:00            1.0   
2                2018-09-04 00:00:00            1.0   
3                2017-12-15 00:00:00            1.0   
4                2018-02-26 00:00:00            1.0   
...                              ...            ...   
113420           2018-03-02 00:00:00            1.0   
113421           2017-09-27 00:00:00            1.0   
113422           2018-02-15 00:00:00            1.0   
113423           2018-02-15 00:00:00            2.0   
113424           2018-04-03 00:00:00            1.0   

                              product_id                         seller_id  \
0       87285b34884572647811a353c7ac498a  3504c0cb71d7fa48d967e0e4c94d59d9   
1       595fac2a385ac33a80bd5114aec74eb8  289cdb325fb7e7f891c38608bf9e0962   
2       aa4383b373c6aca5d8797843e5594415  4869f7a5dfa277a7dca6462dcf3b52b2   
3       d0b61bfb1de832b15ba9d266ca96e5b0  66922902710d126a0e7d26b0e3805106   
4       65266b2da20d04dbe00c5c2d3bb7859e  2c9e548be18521d1c43cde1c582c6de8   
...                                  ...                               ...   
113420  f1d4ce8c6dd66c47bbaa8c6781c2a923  1f9ab4708f3056ede07124aad39a2554   
113421  b80910977a37536adeddd63663f916ad  d50d79cb34e38265a8649c383dcffd48   
113422  d1c427060a0f73f6b889a5c7c61f2ac4  a1043bafd471dff536d0c462352beb48

In [46]:
print_float(df_orders.order_status.isnull().sum())


0.00


In [47]:
df_orders.order_status.value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [104]:
df_orders[(df_orders['order_status']=='delivered')&(df_orders['order_delivered_customer_date']>df_orders['order_estimated_delivery_date'])&(df_orders['order_id'].isin(more_than_one_order))].shape

(629, 8)

In [53]:
df_orders[(df_orders['order_status']=='delivered')&(df_orders['order_delivered_customer_date']<df_orders['order_estimated_delivery_date'])].shape

(88644, 8)

In [57]:

df_orders['order_delivered_customer_date'].apply(lambda x:x.strftime('%m/%d/%Y'))


AttributeError: 'str' object has no attribute 'strftime'